<a href="https://colab.research.google.com/github/FelipeSotoG/Desafio-1-EDA/blob/main/WTABestFirst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WTA


Problema WTA resuelto con Best First Algorithm
Initial state es ingresado manualmente.

In [1]:
from numpy.matrixlib.defmatrix import matrix
import numpy as np
import itertools
#formato de los estados
class State:
  id_iter = itertools.count()
  def __init__(self, matrix, weapons, prob, value):
    self.id = next(State.id_iter)
    self.matrix = matrix
    self.weapons = weapons
    self.prob = prob
    self.value = value
  def __lt__(self, other):
    return self.id < other.id
#matriz "vacía"
initial_state = State([[0,0,0],\
                       [0,0,0],\
                       [0,0,0]],
                       [5,2,1],
                      [[0.3,0.2,0.5],\
                       [0.1,0.6,0.5],\
                       [0.4,0.5,0.4]],
                       [5,10,20] )

class Action:
  def __init__(self,i,j):
    self.i=i; self.j=j

N=3 #número de filas/columnas

La trancision es solo agregar uno a un item de la matriz

In [2]:
def transition(state, action):
  state.matrix[action.i][action.j] +=1 

Para saber **si un estado es final**, tenemos que ver si se usaron todas las armas y todos los objetidos fueron "atacados".

In [3]:
def column_sum(lst):  
     return [sum(i) for i in zip(*lst)]

In [4]:
def is_final_state(state):
  i=0
  for x in column_sum(state.matrix):
    if x == 0: return False
    i+=1
  i=0
  for x in state.matrix:
    if sum(x) != state.weapons[i]: return False
    i+=1
  return True

El estado es calido cuando no hay mas armas en una fila de las que deberian.

In [5]:
def is_valid(state):
  i=0
  for x in state.matrix:
    if sum(x) > state.weapons[i]: return False
    i+=1
  return True

Las acciones validas para este problema son toda la matriz.

In [6]:
def get_valid_actions(state):
  valid_actions = []
  for i in range(N):
    for j in range(N):
      valid_actions.append(Action(i,j))
  return valid_actions

## Resolución del problema

Ya modelado pordemos hacer la huristica que ocuparemos, la cual es la funcion objetivo.

Luego podemos implementar el algoritmo Best First.

V1 con una version de la heuristica

In [22]:
import numpy as np

def heuristic_eval(state):
    minsum=0
    for j in range(N):
      arr=[]
      for i in range(N):
        if state.matrix[i][j] != 0: arr.append(1-pow(state.prob[i][j],state.matrix[i][j]))
      minsum+= state.value[j]*np.prod(arr) 
    return minsum

In [36]:
import copy

from queue import PriorityQueue

def bfs(initial_state):
  q = PriorityQueue()
  q.put( (-1, copy.deepcopy(initial_state)) )
  iters = 0
  while not q.empty():
    iters += 1
    elem=q.get()
    state = elem[1]

    if is_valid(state) == False: continue #se descarta el estado

    if is_final_state(state): return state,iters #se encontró una solución!

    actions = get_valid_actions(state)

    for action in actions:
      new_state = copy.deepcopy(state)
      transition(new_state,action)
      q.put((heuristic_eval(new_state), new_state))


In [35]:
state,iters=bfs(initial_state)
print(state.matrix)
print(heuristic_eval(state))
print("iteraciones:", iters)

[[3, 1, 1], [0, 1, 1], [1, 0, 0]]
11.119
iteraciones: 19


El problema con esto, en relacion al DFS es que la heuristica es mala para decidir caminos al inicio.

V2 con una nueva version de la heuristica

In [51]:
import numpy as np

def heuristic_eval2(state):
    minsum=0
    for j in range(N):
      arr=[]
      for i in range(N):
        if state.matrix[i][j] != 0: arr.append(1-pow(state.prob[i][j],state.matrix[i][j]))
      minsum+= state.value[j]*np.prod(arr) 
    i=0
    for x in state.matrix:
      minsum+=(state.weapons[i]-np.sum(x))*10
      i+=1
    print(minsum)
    return minsum

In [49]:
import copy

from queue import PriorityQueue

def bfs(initial_state):
  q = PriorityQueue()
  q.put( (-1, copy.deepcopy(initial_state)) )
  iters = 0
  while not q.empty():
    iters += 1
    elem=q.get()
    state = elem[1]

    if is_valid(state) == False: continue #se descarta el estado

    if is_final_state(state): return state,iters #se encontró una solución!

    actions = get_valid_actions(state)

    for action in actions:
      new_state = copy.deepcopy(state)
      transition(new_state,action)
      q.put((heuristic_eval2(new_state)+100-iters, new_state))

In [52]:
state,iters=bfs(initial_state)
print(state.matrix)
print(heuristic_eval(state))
print("iteraciones:", iters)

103.5
103.0
95.0
104.5
99.0
95.0
103.0
100.0
97.0
83.5
83.0
90.0
84.5
79.0
80.0
83.0
80.0
81.0
67.5
68.2
74.0
68.5
71.4
64.0
67.0
67.0
65.0
52.5
53.2
56.5
53.5
56.4
56.5
52.0
52.0
52.0
41.1
41.2
44.5
41.7
44.4
44.5
43.2
40.0
40.0
31.73
30.3
33.6
30.89
33.5
33.6
31.939999999999998
29.1
29.1
20.93
20.939999999999998
22.8
20.09
22.22
22.799999999999997
21.14
18.7
18.3
11.119
11.57
13.430000000000001
10.657
12.850000000000001
13.43
12.021999999999998
9.330000000000002
8.93
[[3, 1, 1], [0, 1, 1], [1, 0, 0]]
11.119
iteraciones: 19
